# Spatial Resolution & Swath vs Focal Length Trade
Tradeoff between spatial resolution and swath width for a given range of foreoptic focal lengths.

In [1]:
# stdlib
from pathlib import Path

# external
import astropy.units as unit
import numpy as np
import pandas as pd
import plot
from IPython.display import display

# project
from architect.libs import utillib
from architect.systems.optical import foreoptics, masks, sensors, spectrometers

## Setup
Run this section onwards when a parameter is updated.

In [2]:
output_dir = Path("output/spatial_resolution_&_swath_vs_focal_length_trade/")
output_dir.mkdir(parents=True, exist_ok=True)

### Parameters

In [5]:
# region constants
orbital_altitude = 550 * unit.km
foreoptic_diameter = 44 * unit.mm
skew_angle = np.array([0, 0]) * unit.deg
slit_size = np.array([3, 1]) * unit.mm
target_wavelength = 1300 * unit.nm
# endregion

# region variables
focal_length = np.arange(start=10, stop=300, step=10) * unit.mm
# endregion

# region vectorization
parameter_space_shape = (focal_length.size, slit_size.size)
focal_length = utillib.orient_and_broadcast(
    a=focal_length, dim=0, shape=parameter_space_shape
)
# endregion

### Instantiation

In [6]:
# region components
sensor = sensors.TauSWIR()
foreoptic = foreoptics.Foreoptic(diameter=foreoptic_diameter, focal_length=focal_length)
slit = masks.RectSlit(size=slit_size)
# endregion

# region systems
payload = spectrometers.HyperspectralImager(
    sensor=sensor, foreoptic=foreoptic, slit=slit
)
# endregion

## Pipeline

In [7]:
sensor_spatial_resolution = payload.get_spatial_resolution(
    wavelength=target_wavelength,
    target_distance=orbital_altitude,
    skew_angle=skew_angle[0],
).to(unit.m)

display(
    f"Sensor-limited spatial resolution test shape: {sensor_spatial_resolution.shape}"
)

swath = payload.get_swath(altitude=orbital_altitude, skew_angle=skew_angle)
display(f"Swath shape: {swath.shape}")

UnitConversionError: Can only apply 'maximum' function to quantities with compatible dimensions

## Plots

In [6]:
focal_length_label = f"Focal Length [{focal_length.unit}]"
sensor_spatial_resolution_label = (
    f"Spatial Resolution [{sensor_spatial_resolution.unit}]"
)
swath_label_x = f"Swath (across-track) [{swath.unit}]"
swath_label_y = f"Swath (along-track) [{swath.unit}]"


data = {
    focal_length_label: focal_length[:, 0],
    sensor_spatial_resolution_label: sensor_spatial_resolution[:, 0],
    swath_label_x: swath[:, 0],
    swath_label_y: swath[:, 1],
}

df = pd.DataFrame.from_dict(data=data)

graph = plot.line(
    df=df,
    x=focal_length_label,
    y=[
        sensor_spatial_resolution_label,
        swath_label_x,
        swath_label_y,
    ],
    title=None,
    dark=True,
)

df.to_csv(output_dir / "data.csv", index=False)
plot.save(
    fig=graph, name="spatial_resolution_&_swath_vs_focal_length_trade", path=output_dir
)

graph.show()
display(df)
display(payload)

,Focal Length [mm],Spatial Resolution [m],Swath (across-track) [km],Swath (along-track) [km]
0,10.0,825.000000,165.000000,55.000000
1,20.0,412.500000,82.500000,27.500000
2,30.0,275.000000,55.000000,18.333333
3,40.0,206.250000,41.250000,13.750000
4,50.0,165.000000,33.000000,11.000000
5,60.0,137.500000,27.500000,9.166667
6,70.0,117.857143,23.571429,7.857143
7,80.0,103.125000,20.625000,6.875000
8,90.0,91.666667,18.333333,6.111111
9,100.0,82.500000,16.500000,5.500000


,Value,Units
foreoptic,Foreoptic,None
slit,RectSlit,None
diffractor,None,None
sensor,TauSWIR,None
components,list [4],None
